1.  Напишите программу, которая создает нить. Родительская и вновь созданная нити должны распечатать десять строк текста.

In [ ]:
import threading

# Функция, которую будет выполнять нить
def print_ten_lines():
    for i in range(10):
        print("This is line", i+1)

# Создание новой нити
thread = threading.Thread(target=print_ten_lines)

# Запуск нити
thread.start()

# Главная нить также будет печатать десять строк текста
for i in range(10):
    print("This is line", i+1)

2. Напишите простой эхо-сервер, использующий неблокирующие сокеты и клиент к нему.

Сервер:

In [ ]:
import socket

# Создание сокета
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Установка опции сокета для неблокирующего режима
server_socket.setblocking(False)

# Привязка сокета к адресу и порту
server_socket.bind(('localhost', 8888))

# Прослушивание подключений
server_socket.listen(5)
print("Server is listening...")

# Список подключенных клиентов
clients = []

# Функция для обработки входящих соединений и данных
def handle_connections():
    while True:
        # Принятие входящего соединения
        try:
            client_socket, client_address = server_socket.accept()
            print(f"Connection from {client_address}")
            # Добавление клиента в список
            clients.append(client_socket)
        except BlockingIOError:
            pass

        # Обработка входящих данных от клиентов
        for client in clients:
            try:
                data = client.recv(1024)
                if data:
                    print(f"Received: {data.decode()}")
                    # Отправка обратно данных клиенту
                    client.sendall(data)
                else:
                    # Если клиент закрыл соединение, удаляем его из списка
                    clients.remove(client)
                    client.close()
            except BlockingIOError:
                pass

# Запуск функции для обработки соединений в отдельном потоке
import threading
threading.Thread(target=handle_connections).start()

Клиент:

In [ ]:
import socket

# Создание сокета
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Подключение к серверу
client_socket.connect(('localhost', 8888))

# Отправка данных серверу
client_socket.sendall(b"Hello, server!")

# Получение данных от сервера
data = client_socket.recv(1024)
print("Received from server:", data.decode())

# Закрытие соединения
client_socket.close()

3. Напишите простой многопоточный загрузчик URL. Список URL скрипт принимает как аргументы командной строки.

In [ ]:
import threading
import requests
import sys

# Функция для загрузки URL
def load_url(url):
    try:
        response = requests.get(url)
        print(f"Loaded {url} with status code {response.status_code}")
    except Exception as e:
        print(f"Error loading {url}: {e}")

# Функция для создания и запуска нитей для загрузки URL
def start_threads(urls):
    threads = []
    for url in urls:
        thread = threading.Thread(target=load_url, args=(url,))
        threads.append(thread)
        thread.start()
    # Ожидание завершения всех нитей
    for thread in threads:
        thread.join()

if __name__ == "__main__":
    # Проверка наличия аргументов командной строки
    if len(sys.argv) < 2:
        print("Usage: python downloader.py url1 url2 url3 ...")
        sys.exit(1)

    # Получение списка URL из аргументов командной строки
    urls = sys.argv[1:]

    # Запуск загрузки URL в многопоточном режиме
    start_threads(urls)